In [142]:
import bs4
import os
import pandas as pd

from datetime import datetime, timedelta

def is_stream_span(el):
    target_class_list = 'TableCell__TableCellElement-sc-1nn7cfv-0 kJgiFu styled__RightTableCell-sc-135veyd-4 kGfYTK'.split(' ')
    if not el.has_attr('class'):
        return False
    return set(el['class']) == set(target_class_list) 

source_dir = "/home/aldakata/Projects/Tuebingen/Data Literacy/project/spotify_data/"
source_file = "global_2023-01-05.html"

def extract_data(source_dir, source_file):
    """Function to extract data from the html files"""
    with open(f"{source_dir}/{source_file}", "r") as input_file:
        data = input_file.read()
    # Turn the content into soup
    soup = bs4.BeautifulSoup(data, features="html.parser")

    # Find all the rows in the html
    rows = soup.findAll("tr")
    df_rows = []

    for i, row in enumerate(rows[1:]):
        row_url = row.find('a', class_='Button-me270r-0', href=True)['href']
        row_data = {"spotify_url": row_url.replace('https://open.spotify.com/track/', '')}
        week_start = source_file.split(".")[0].split("_")[-1]
        row_data["week_start"] = week_start
        week_start.split("-")
        row_data["streams"] = int(row.find_all(is_stream_span)[0].get_text().replace(',', ''))
        row_data['country'] = source_file.split(".")[0].split("_")[0]
        row_data['ranking'] = i + 1
        df_rows.append(row_data)

    return pd.DataFrame(df_rows)

In [143]:
df = extract_data(source_dir, source_file)
df

,spotify_url,week_start,streams,country,ranking
0,https://open.spotify.com/track/1Qrg8KqiBpW07V7...,2023-01-05,41474366,global,1
1,https://open.spotify.com/track/3nqQXoyQOWXiESF...,2023-01-05,33132175,global,2
2,https://open.spotify.com/track/4Dvkj6JhhA12EX0...,2023-01-05,30998084,global,3
3,https://open.spotify.com/track/4uUG5RXrOk84mYE...,2023-01-05,30704256,global,4
4,https://open.spotify.com/track/2dHHgzDwk4BJdRw...,2023-01-05,28092232,global,5
...,...,...,...,...,...
195,https://open.spotify.com/track/4tYFy8ALRjIZvnv...,2023-01-05,6782777,global,196
196,https://open.spotify.com/track/37F7E7BKEw2E4O2...,2023-01-05,6775767,global,197
197,https://open.spotify.com/track/5g7sDjBhZ4I3gcF...,2023-01-05,6758678,global,198
198,https://open.spotify.com/track/3rb0tMq42Wfgguc...,2023-01-05,6748442,global,199


In [152]:
list_df = []

for i, file_path in enumerate(os.listdir(source_dir)):
    if any([c in file_path for c in ['global', 'es', 'de', 'ie']]):
        print(i, file_path)
        list_df.append(extract_data(source_dir, file_path))

global_2022-12-15.html
de_2022-11-03.html
de_2022-02-24.html
de_2022-04-21.html
de_2022-01-27.html
global_2022-11-10.html
es_2022-04-07.html
ie_2022-08-18.html
es_2022-06-30.html
ie_2022-03-10.html
ie_2022-12-29.html
ie_2022-02-17.html
de_2022-10-13.html
global_2022-06-16.html
global_2022-11-03.html
ie_2022-12-22.html
de_2022-06-30.html
global_2022-02-24.html
ie_2022-03-31.html
global_2022-07-07.html
es_2022-07-21.html
de_2022-03-10.html
global_2022-09-08.html
global_2022-03-31.html
es_2022-05-19.html
es_2022-10-20.html
de_2022-05-26.html
ie_2022-12-15.html
es_2022-04-14.html
ie_2022-05-12.html
de_2022-09-08.html
global_2022-08-04.html
de_2022-07-14.html
es_2022-02-17.html
es_2022-11-17.html
global_2022-06-23.html
es_2022-03-03.html
es_2022-07-07.html
de_2022-03-17.html
de_2022-03-03.html
global_2022-06-30.html
de_2022-05-05.html
global_2022-02-10.html
ie_2022-06-02.html
de_2022-12-29.html
global_2022-12-08.html
es_2022-09-08.html
de_2022-04-07.html
es_2022-01-20.html
ie_2022-04-28.htm

In [153]:
target_pickle = 'toy_sample'
df = pd.concat(list_df)
df.to_pickle(f"{source_dir}/{target_pickle}.pkl")  

In [172]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = "634a4ade4666411689ac574f82e022df"
secret = "c4d5808135ac4908aeef898447a625af"

client_credentials_manager = SpotifyClientCredentials(
    client_id=cid, client_secret=secret
)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_track_features(id):
    metadata = sp.track(id)
    features = sp.audio_features(id)

    # metadata
    name = metadata["name"]
    album = metadata["album"]["name"]
    artist = metadata["album"]["artists"][0]["name"]
    release_date = metadata["album"]["release_date"]
    length = metadata["duration_ms"]
    popularity = metadata["popularity"]

    # audio features
    acousticness = features[0]["acousticness"]
    danceability = features[0]["danceability"]
    energy = features[0]["energy"]
    instrumentalness = features[0]["instrumentalness"]
    liveness = features[0]["liveness"]
    loudness = features[0]["loudness"]
    speechiness = features[0]["speechiness"]
    tempo = features[0]["tempo"]
    time_signature = features[0]["time_signature"]
    url = id

    track = [
        name,
        album,
        artist,
        release_date,
        length,
        popularity,
        danceability,
        acousticness,
        energy,
        instrumentalness,
        liveness,
        loudness,
        speechiness,
        tempo,
        time_signature,
        url,
    ]
    return track


tracks = []
for i, url in enumerate(temp_urls):
    print(f'Processing... {i} {url}')
    track = get_track_features(url)
    tracks.append(track)



df_api = pd.DataFrame(
    tracks,
    columns=[
        "name",
        "album",
        "artist",
        "release_date",
        "length",
        "popularity",
        "danceability",
        "acousticness",
        "energy",
        "instrumentalness",
        "liveness",
        "loudness",
        "speechiness",
        "tempo",
        "time_signature",
        "spotify_url",
    ],
)



Processing... 0 0bYg9bo50gSsH3LtXe2SQn
Processing... 1 2FRnf9qhLbvw8fu4IBXx78
Processing... 2 1Qrg8KqiBpW07V7PNxwwwL
Processing... 3 2EjXfH91m7f8HiJN1yQg97
Processing... 4 3nqQXoyQOWXiESFLlDF1hG
Processing... 5 2dHHgzDwk4BJdRwy9uXhTO
Processing... 6 7vQbuQcyTflfCIOu3Uzzya
Processing... 7 4Dvkj6JhhA12EX05fT7y2e
Processing... 8 1rv46mRwDqMEhOBZ7vODg3
Processing... 9 5ww2BF9slyYgNOk37BlC4u
Processing... 10 0V3wPSX9ygBnCm8psDIegu
Processing... 11 5Y35SjAfXjjG0sFQ3KOxmm
Processing... 12 0lizgQ7Qw35od7CYaoMBZb
Processing... 13 2mnXxnrX5vCGolNkaFvVeM
Processing... 14 4uUG5RXrOk84mYEfFvj3cK
Processing... 15 2tTmW7RDtMQtBk7m2rYeSw
Processing... 16 1IcR6RlgvDczfvoWJSSY2A
Processing... 17 1bDbXMyjaUIooNwFE9wn0N
Processing... 18 2uFaJJtFpPDc5Pa95XzTvg
Processing... 19 6Sq7ltF9Qa7SNFBsV5Cogx
Processing... 20 2wSTnntOPRi7aQneobFtU4
Processing... 21 0QHEIqNKsMoOY5urbzN48u
Processing... 22 3qCeq0rWK67SoBDgFd2sId
Processing... 23 2CSRrnOEELmhpq8iaAi9cd
Processing... 24 1IHWl5LamUGEuP4ozKQSXZ
Processing

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [157]:
url = 'https://open.spotify.com/track/3nqQXoyQOWXiESF'
url.replace('https://open.spotify.com/track/', '')

'3nqQXoyQOWXiESF'

In [170]:
temp_urls = set([e.replace('https://open.spotify.com/track/', '') for e in df[df['week_start'].isin(['2022-12-15', '2022-08-25'])].spotify_url])
len(temp_urls)

1600

In [168]:
len(df[df['week_start'].isin(['2022-12-15', '2022-08-25'])])

1600

In [181]:
df_temp = pd.DataFrame(df[df['week_start'].isin(['2022-12-15', '2022-08-25'])][:1285])

In [210]:
df_api = pd.DataFrame(
    tracks,
    columns=[
        "name",
        "album",
        "artist",
        "release_date",
        "length",
        "popularity",
        "danceability",
        "acousticness",
        "energy",
        "instrumentalness",
        "liveness",
        "loudness",
        "speechiness",
        "tempo",
        "time_signature",
        "spotify_url",
    ],
)

In [213]:
df_api.spotify_url, df_temp.spotify_url

(0       0bYg9bo50gSsH3LtXe2SQn
 1       2FRnf9qhLbvw8fu4IBXx78
 2       1Qrg8KqiBpW07V7PNxwwwL
 3       2EjXfH91m7f8HiJN1yQg97
 4       3nqQXoyQOWXiESFLlDF1hG
                  ...          
 1280    6Y46tOTRhkBamosyuWa6YX
 1281    6wtZPYBIXUvCpXwVjMCJBf
 1282    2uky0VCI2EZni0nArOPznM
 1283    2JPLbjOn0wPCngEot2STUS
 1284    0ynhqjP6gIxoaGTWibGhjc
 Name: spotify_url, Length: 1285, dtype: object,
 0     0bYg9bo50gSsH3LtXe2SQn
 1     2FRnf9qhLbvw8fu4IBXx78
 2     1Qrg8KqiBpW07V7PNxwwwL
 3     2EjXfH91m7f8HiJN1yQg97
 4     3nqQXoyQOWXiESFLlDF1hG
                ...          
 80    6Y46tOTRhkBamosyuWa6YX
 81    6wtZPYBIXUvCpXwVjMCJBf
 82    2uky0VCI2EZni0nArOPznM
 83    2JPLbjOn0wPCngEot2STUS
 84    0ynhqjP6gIxoaGTWibGhjc
 Name: spotify_url, Length: 1285, dtype: object)

In [215]:
df_api.columns

Index(['name', 'album', 'artist', 'release_date', 'length', 'popularity',
       'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature',
       'spotify_url'],
      dtype='object')

In [216]:
df_temp.columns

Index(['spotify_url', 'week_start', 'streams', 'country', 'ranking'], dtype='object')

In [ ]:
pd.concat([df_api, df_temp])

In [233]:
df_joined  = df_api.set_index('spotify_url').join(df_temp.set_index('spotify_url') )
df_joined.head(30)

,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,week_start,streams,country,ranking
spotify_url,,,,,,,,,,,,,,,,,,,
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-12-15,80771,ie,97
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,120868,ie,24
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,6368731,global,156
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-12-15,80771,ie,97
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,120868,ie,24
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,6368731,global,156
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-12-15,80771,ie,97
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,120868,ie,24
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,6368731,global,156


In [234]:
len(df_joined), len(df_temp), len(df_api)

(2863, 1285, 1285)